In [42]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

class CustomDataset():
    """Custom dataset Loader for California Housing
    
    : Dataset <tuple>: tuple of two numpy arrays where index 0 contains the 
                    features (fetch_california_housing.data) and index 1 
                    contains the target values (fetch_california_housing.target)
     """
    def __init__(self, Dataset:tuple):
        """Separate the input"""
        self.data = Dataset[0]
        self.target = Dataset[1]

    def __len__(self):
        """Returns the lenght of the dataset"""
        return len(self.data)

    def __getitem__(self, idx):
        """returns at index idx"""
        return self.data[idx], self.target[idx]

In [43]:
# Data Preparation


housing = fetch_california_housing()
print(housing.data.shape)
print(housing.target.shape)


# Slicing data
## Train Set
my_scaler = StandardScaler()
scaled_data= my_scaler.fit_transform(X=housing.data, y=housing.target)

print(type(scaled_data))


(20640, 8)
(20640,)
<class 'numpy.ndarray'>


In [44]:


data_train = torch.from_numpy(np.float32(scaled_data[:1000]))
target_train = torch.from_numpy(np.float32(housing.target[:1000]))

print(data_train.shape)
print(target_train.shape)

train_set = (data_train,target_train)

## Dev Set
data_dev = torch.from_numpy(np.float32(scaled_data[1000:1100]))
target_dev = torch.from_numpy(np.float32(housing.target[1000:1100]))

print(data_dev.shape)
print(target_dev.shape)
dev_set = (data_dev,target_dev)


custom_train = CustomDataset(train_set)
train_dataloader = DataLoader(custom_train, batch_size=1)

custom_train = CustomDataset(dev_set)
dev_dataloader = DataLoader(custom_train, batch_size=1)

torch.Size([1000, 8])
torch.Size([1000])
torch.Size([100, 8])
torch.Size([100])


In [47]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            #correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Avg MSE loss : {test_loss:>8f} \n")

In [50]:
model = NeuralNetwork()

loss_fn = nn.MSELoss()
learning_rate = .001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(dev_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss: 21.418568  [    0/ 1000]
loss: 2.878118  [  100/ 1000]
loss: 1.231503  [  200/ 1000]
loss: 0.422587  [  300/ 1000]


/home/lucasfl/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss: 0.557020  [  400/ 1000]
loss: 0.409181  [  500/ 1000]
loss: 0.146185  [  600/ 1000]
loss: 0.543115  [  700/ 1000]
loss: 0.002199  [  800/ 1000]
loss: 0.058365  [  900/ 1000]
Avg MSE loss : 1.184848 

Epoch 2
-------------------------------
loss: 0.179446  [    0/ 1000]
loss: 1.152719  [  100/ 1000]
loss: 0.496212  [  200/ 1000]
loss: 0.033392  [  300/ 1000]
loss: 0.348483  [  400/ 1000]
loss: 0.146327  [  500/ 1000]
loss: 0.075511  [  600/ 1000]
loss: 0.467657  [  700/ 1000]
loss: 0.000160  [  800/ 1000]
loss: 0.036381  [  900/ 1000]
Avg MSE loss : 1.230074 

Epoch 3
-------------------------------
loss: 0.236759  [    0/ 1000]
loss: 1.034773  [  100/ 1000]
loss: 0.419639  [  200/ 1000]
loss: 0.019994  [  300/ 1000]
loss: 0.315251  [  400/ 1000]
loss: 0.127008  [  500/ 1000]
loss: 0.079516  [  600/ 1000]
loss: 0.484733  [  700/ 1000]
loss: 0.000134  [  800/ 1000]
loss: 0.034266  [  900/ 1000]
Avg MSE loss : 1.309016 

Epoch 4
-------------------------------
loss: 0.244967  [    0